In [ ]:
import os,urllib.request
import numpy as np,codecs
s=os.getcwd()
s=s+"\\mnist\\"

files=os.listdir(s)

def gint(b):
    return int(codecs.encode(b,'hex'),16)

dic={}

for file in files:
    if file.endswith('ubyte'):
        print('Reading',file)
        with open(s+file,'rb') as f:
            data=f.read()
            mnam=gint(data[:4])
            nosam=gint(data[4:8])
            if mnam==2049:
                cat='label'
                parsed=np.frombuffer(data,dtype=np.uint8,offset=8)
                parsed=parsed.reshape(nosam)
                print(parsed)
            elif mnam==2051:
                cat='Image'
                nor=gint(data[8:12])
                noc=gint(data[12:16])
                parsed=np.frombuffer(data,dtype=np.uint8,offset=16)
                parsed=parsed.reshape(nosam,nor,noc)
            if nosam==10000:
                e='test'
            else:
                e='train'
            dic[cat+'_'+e]=parsed
#print(dic['Image_test'][0])   

In [ ]:
import cv2 as cv
#cv.imwrite('temp.jpg',dic['Image_test'][0])
cv.imshow('Image',dic['Image_test'][1000])
#print(type(int(dic['Image_test'][9])))
cv.waitKey(0);
cv.destroyAllWindows();

In [ ]:
print(dic['Image_test'][1000].dtype)

In [ ]:
import math,random
z={}
for i in range(len(dic['Image_test'])):
    z[i]=np.zeros((56,56),dtype='uint8')
    a=1
    im=cv.resize(dic['Image_test'][i],((math.floor(a*28),math.floor(a*28))))
    e=56-math.floor(a*28)
    w=math.floor(e*np.random.random((1)))
    h=math.floor(e*np.random.random((1)))
    z[i][w:w+56-e,h:h+56-e]=im

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
for i in range(1000,1006):
    plt.imshow(z[i])

In [ ]:
import math
typ=['train','test']
for t in typ:
    x=[]
    y=[]
    for i in range(len(dic['Image_'+t])):
        z=np.zeros((1,56,56),dtype='uint8')
        a=1
        l=dic['label_'+t][i]
        im=cv.resize(dic['Image_'+t][i],((math.floor(a*28),math.floor(a*28))))
        e=56-math.floor(a*28)
        w=math.floor(e*np.random.random((1)))
        h=math.floor(e*np.random.random((1)))
        z[0,w:w+56-e,h:h+56-e]=im
        x.append(z)
        y.append(l)
    np.save(t+'_data.npy',np.asarray(x))
    np.save(t+'_label.npy',np.asarray(y))


In [ ]:
dic['label_train'][0]

In [3]:
import numpy as np
import cv2
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import math
class GridDataset(Dataset):
  def __init__(self,type):
    s = os.getcwd()
    self.x=[]
    self.y=[]
    self.x=np.load(type+"_data.npy", allow_pickle=True)
    self.y=np.load(type+'_label.npy',allow_pickle=True)
    self.x=torch.from_numpy(self.x/255).float()
    self.y=torch.from_numpy(self.y).float()

  def __getitem__(self,index):
    return self.x[index], self.y[index]
  def __len__(self):
    return self.x.shape[0]

In [4]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, 1,1)
        self.conv2 = nn.Conv2d(64, 64, 3, 1,1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(12544*4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        target=target.long()
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            target=target.long()
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            data, target = data.to("cpu"), target.to("cpu")

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():
    batch_size=64
    test_batch_size=1000
    epochs=3
    lr=1
    gamma=0.7
    no_cuda=False
    seed=1
    log_interval=10
    save_model=False
    
    use_cuda = not no_cuda and torch.cuda.is_available()

    torch.manual_seed(seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
    
    data_test=GridDataset('test')
    data_train=GridDataset('train')
    train_loader=torch.utils.data.DataLoader(data_train,batch_size=batch_size,shuffle=True, **kwargs)
    test_loader=torch.utils.data.DataLoader(data_test,batch_size=test_batch_size,shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train(log_interval, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")
        
    return model


am=main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309844
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.298390
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.316203
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.303821
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.301211
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.267508
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.282256
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.235485
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.221292
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.187207
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.076581
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.335712
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.119597
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.127778
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.900096
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.937241
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.254881
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.779628
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.882846
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.479203
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.768881
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.670069
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.400160
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.643294
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.443497
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.607423
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.641202
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.519663
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.495174
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.727258
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.529977
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.707204
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.538192
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.682305
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.465914
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.610289
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.534026
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.468615
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.716461
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.613722


In [ ]:
use_cuda =torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
te=1
if te==1:
    data_test=GridDataset('test')
    bm=torch.utils.data.DataLoader(data_test,batch_size=20,shuffle=True)
for data,label in bm:
    am.eval()
    data,label=data.to(device), label.to(device)
    x1=am.conv1(data)
    x1=F.relu(x1) # The first covolutional layer 
    x2 = am.conv2(x1)
    x2 = F.relu(x2) #The second convolutional layer
    break
x1=x1.to('cpu').detach().numpy()
x2=x2.to('cpu').detach().numpy()

In [5]:
co1=np.zeros((3136,3136))
for i in range(3136):
    for j in range(i,3136):
        a=math.floor(i/56)
        b=i%56
        if i==j:
            co1[i][i]=0
        else:
            a1=math.floor(j/56)
            b1=j%56
            q=math.sqrt((a-a1)*(a-a1)+(b-b1)*(b-b1))
            co1[i][j]=q/77.78
            co1[j][i]=q/77.78

In [ ]:
import ot
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
g=20
va1=np.zeros((64,g,g))
for i in range(g):
    for j in range(i,g):
        if i==j:
            va1[:,i,j]=0
        else:
            e=x1[i].reshape((64,56,56)).astype('float64')
            f=x1[j].reshape((64,56,56)).astype('float64')
            print(i,j);
            for k in range(64):
                dis=ot.sinkhorn2(e[k].reshape((3136))/(np.sum(e[k].reshape((3136)),axis=0)),f[k].reshape((3136))/(np.sum(f[k].reshape((3136)),axis=0)),co1,10^-3);
                va1[k][i][j]=dis
                va1[k][j][i]=dis
va2=np.zeros((64,g,g))
for i in range(g):
    for j in range(i,g):
        if i==j:
            va2[:,i,j]=0
        else:
            print(i,j)
            e=x2[i].reshape((64,56,56)).astype('float64')
            f=x2[j].reshape((64,56,56)).astype('float64')
            
            for k in range(64):
                dis=ot.sinkhorn2(e[k].reshape((3136))/(np.sum(e[k].reshape((3136)),axis=0)),f[k].reshape((3136))/(np.sum(f[k].reshape((3136)),axis=0)),co1,10^-3);
                va2[k][i][j]=dis
                va2[k][j][i]=dis
va1=np.sum(va1,axis=0)/64 #averaging it between the number of channels
va2=np.sum(va2,axis=0)/64

In [ ]:

a=np.zeros((10,10))
b=np.zeros((10,10))
for i in range(g):
    for j in range(i,g):
        a[int(label[j])][int(label[i])]+=va1z[int(label[j])][int(label[i])]
        b[int(label[j])][int(label[i])]+=1
        if label[i]!=label[j]:
            a[int(label[i])][int(label[j])]+=va1z[int(label[i])][int(label[j])]
            b[int(label[i])][int(label[j])]+=1
z1=a/b # the average of these distances
a=np.zeros((10,10))
b=np.zeros((10,10))
for i in range(g):
    for j in range(i,g):
        a[int(label[j])][int(label[i])]+=va2z[int(label[j])][int(label[i])]
        b[int(label[j])][int(label[i])]+=1
        if label[i]!=label[j]:
            a[int(label[i])][int(label[j])]+=va2z[int(label[i])][int(label[j])]
            b[int(label[i])][int(label[j])]+=1
z2=a/b

In [ ]:

x=[0,1,2,3,4,5,6,7,8,9]
y=np.zeros((10))
for i in range(16):
    y[int(label[i])]+=1
plt.bar(x,y)
plt.title('Distribution of characters in the batch')
plt.show()

In [ ]:
for i in range(10):
    plt.plot(x,z1[i])
    plt.title('exact OT distances of character '+str(i)+' at the first conv layer')
    plt.show()
    plt.plot(x,z2[i])
    plt.title('exact OT distances of character '+str(i)+' at the second conv layer')
    plt.show()

In [ ]:
import ot
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
g=16
va1z=np.zeros((64,g,g))
for i in range(g):
    for j in range(i,g):
        if i==j:
            va1z[:,i,j]=0
        else:
            e=x1[i].reshape((64,56,56)).astype('float64')
            f=x1[j].reshape((64,56,56)).astype('float64')
            print(i,j);
            for k in range(64):
                dis=ot.emd2(e[k].reshape((3136))/(np.sum(e[k].reshape((3136)),axis=0)),f[k].reshape((3136))/(np.sum(f[k].reshape((3136)),axis=0)),co1);
                va1z[k][i][j]=dis
                va1z[k][j][i]=dis
va2z=np.zeros((64,g,g))
for i in range(g):
    for j in range(i,g):
        if i==j:
            va2z[:,i,j]=0
        else:
            print(i,j)
            e=x2[i].reshape((64,56,56)).astype('float64')
            f=x2[j].reshape((64,56,56)).astype('float64')
            
            for k in range(64):
                dis=ot.emd2(e[k].reshape((3136))/(np.sum(e[k].reshape((3136)),axis=0)),f[k].reshape((3136))/(np.sum(f[k].reshape((3136)),axis=0)),co1);
                va2z[k][i][j]=dis
                va2z[k][j][i]=dis
va1z=np.sum(va1z,axis=0)/64 #averaging it between the number of channels
va2z=np.sum(va2z,axis=0)/64

In [ ]:
va1z

In [ ]:
va2z

In [19]:
def distances(x1,x2,co1,l):
    m=np.zeros((1,64,3136,3136))
    va1=np.zeros((64,2,2))
    z=0
    for i in range(2):
        for j in range(i,2):
            if i==j:
                va1[:,i,j]=0
            else:
                e=x1[i].reshape((64,56,56)).astype('float64')
                f=x1[j].reshape((64,56,56)).astype('float64')

                for k in range(64):
                    sq=e[k].reshape((3136))/(np.sum(e[k].reshape((3136)),axis=0))
                    dis=ot.emd2(sq,f[k].reshape((3136))/(np.sum(f[k].reshape((3136)),axis=0)),co1)
                    mt=ot.emd(e[k].reshape((3136))/(np.sum(e[k].reshape((3136)),axis=0)),f[k].reshape((3136))/(np.sum(f[k].reshape((3136)),axis=0)),co1)
                    m[z,k,:,:]=proc(mt,sq)
                    va1[k][i][j]=dis
                    va1[k][j][i]=dis
                z+=1
    va2=np.zeros((64,2,2))
    va2m1=np.zeros((64,2,2))
    z=0
    for i in range(2):
        for j in range(i,2):
            if i==j:
                va2[:,i,j]=0
            else:
                e=x2[i].reshape((64,56,56)).astype('float64')
                f=x2[j].reshape((64,56,56)).astype('float64')
                #print(1)
                for k in range(64):
                    sq=e[k].reshape((3136))/(np.sum(e[k].reshape((3136))))
                    io=f[k].reshape((3136))/(np.sum(f[k].reshape((3136))))
                    d1=np.matmul((m[z,k,:,:].reshape((3136,3136))).T,sq).astype('float64')
                    dis=ot.emd2(sq,io,co1)
                    mt=ot.emd(sq,io,co1)
                    dism1=ot.emd2(d1,io,co1)
                    va2[k][i][j]=dis
                    va2[k][j][i]=dis
                    va2m1[k][i][j]=dism1
                    va2m1[k][j][i]=dism1
                z+=1
    va1=np.sum(va1,axis=0)/64 #averaging it between the number of channels
    va2=np.sum(va2,axis=0)/64
    va2m1=np.sum(va2m1,axis=0)/64
    l.append((va1,va2,va2m1))

In [20]:
import ot
use_cuda =torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
te=1
i=0
l=[]
if te==1:
    data_test=GridDataset('test')
    bm=torch.utils.data.DataLoader(data_test,batch_size=2,shuffle=True)
for data,label in bm:
    am.eval()
    data,label=data.to(device), label.to(device)
    x1=am.conv1(data)
    x1=F.relu(x1) # The first covolutional layer 
    x2 = am.conv2(x1)
    x2 = F.relu(x2) #The second convolutional layer
    x1=x1.to('cpu').detach().numpy()
    x2=x2.to('cpu').detach().numpy()
    data, label = data.to("cpu"), label.to("cpu")
    if i>=1:
        break
    print(i)
    distances(x1,x2,co1,l)
    i+=1

0


C:\Users\Ayush Shrivastav\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
C:\Users\Ayush Shrivastav\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\Ayush Shrivastav\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
C:\Users\Ayush Shrivastav\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Ayush Shrivastav\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
for i in range(16):
    print(label[i])

In [ ]:
z=data.to("cpu").numpy()

plt.imshow(z[14][0])

In [6]:
def proc(m,eq):
    for i in range(int(eq.size)):
        if eq[i]!=0:
            m[i,:]=m[i,:]/eq[i] 
    return m

In [21]:
a=np.zeros((2,2))
b=np.zeros((2,2))
c=np.zeros((2,2))
for i in range(1):
    a1,b1,c1=l[i]
    a+=a1
    b+=b1
    c+=c1

In [22]:
b

array([[0.        , 0.23504981],
       [0.23504981, 0.        ]])

In [23]:
c

array([[0.        , 0.00162735],
       [0.00162735, 0.        ]])

In [24]:
a

array([[0.        , 0.10181321],
       [0.10181321, 0.        ]])